In [1]:
import requests
import os
import json
import glob
import ast
import pandas as pd

In [2]:
g_core_name="honzuki_test"
g_url_base="http://127.0.0.1:8983/solr/"+g_core_name+"/"

In [3]:
def get_url_add(url_base,is_commit=True):
    commit = "true" if is_commit else "false"
    header={"Content-Type": "text/json"}
    return url_base+"update?commit="+commit,header

In [4]:
add_data = {"id":"AAA","main_text":"test"}


In [6]:
url,header = get_url_add(g_url_base)
url

'http://127.0.0.1:8983/solr/honzuki_test/update?commit=true'

In [18]:
def add_post(url_base,add_data):

    url,header = get_url_add(url_base)
    print(url)
    ret=requests.post(url,headers=header,data=json.dumps([add_data]))
    print(ret)
    print(ret.text)

In [50]:
add_post(g_url_base,{"id":"1234","title":"鈴木","main_text":"骨側一郎"})
add_post(g_url_base,{"id":"123456","title":"山田","main_text":"鈴木一郎"})
add_post(g_url_base,{"id":"1234567","title":"鈴木","main_text":"None"})
add_post(g_url_base,{"id":"123456789","title":"鈴木 鈴木","main_text":"None"})
add_post(g_url_base,{"id":"1234567890","title":"鈴木 鈴木健司","main_text":"None"})

http://127.0.0.1:8983/solr/honzuki_test/update?commit=true
<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":79}}

http://127.0.0.1:8983/solr/honzuki_test/update?commit=true
<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":69}}

http://127.0.0.1:8983/solr/honzuki_test/update?commit=true
<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":71}}

http://127.0.0.1:8983/solr/honzuki_test/update?commit=true
<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":75}}

http://127.0.0.1:8983/solr/honzuki_test/update?commit=true
<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":81}}



In [15]:
# 対象を削除
def delete_post(url_base,delete_id):
    url,header = get_url_add(g_url_base)
    print(url)
    ret=requests.post(url,headers=header,data=json.dumps({"delete":{"id":delete_id}}))
    print(ret)
    print(ret.text)

In [17]:
delete_post(g_url_base,"AAA")

http://127.0.0.1:8983/solr/honzuki_test/update?commit=true
<Response [200]>
{
  "responseHeader":{
    "status":0,
    "QTime":16}}



In [60]:
search_word="*:*"
url=g_url_base+"select?q="+search_word+"&wt=json&indent=true&defType=edismax&qf=title^1.5+main_text"
print(url)
ret=requests.post(url,headers=header)

jsonData = ret.json()["response"]["docs"]
for i in range(len(jsonData)):
    for key in jsonData[i].keys():
        data = jsonData[i][key]
        if type(data)==str:
            data=data[:50]
        if "_version_"==key:
            continue
        print("{0}:{1}".format(key,data))
    print("-----------------------------")
   

http://127.0.0.1:8983/solr/honzuki_test/select?q=*:*&wt=json&indent=true&defType=edismax&qf=title^1.5+main_text
id:1234
title:鈴木
main_text:骨側一郎
-----------------------------
id:123456
title:山田
main_text:鈴木一郎
-----------------------------
id:1234567
title:鈴木
main_text:None
-----------------------------
id:123456789
title:鈴木 鈴木
main_text:None
-----------------------------
id:1234567890
title:鈴木 鈴木健司
main_text:None
-----------------------------


In [61]:
search_word="鈴木 一郎"
url=g_url_base+"select?q="+search_word+"&wt=json&indent=true&defType=edismax&qf=title^1.5+main_text&fl=*,score"
print(url)
ret=requests.post(url,headers=header)

jsonData = ret.json()["response"]["docs"]
for i in range(len(jsonData)):
    for key in jsonData[i].keys():
        data = jsonData[i][key]
        if type(data)==str:
            data=data[:50]
        if "_version_"==key:
            continue
            
        print("{0}:{1}".format(key,data))
    print("-----------------------------")
   

http://127.0.0.1:8983/solr/honzuki_test/select?q=鈴木 一郎&wt=json&indent=true&defType=edismax&qf=title^1.5+main_text&fl=*,score
id:1234
title:鈴木
main_text:骨側一郎
score:2.7510214
-----------------------------
id:123456
title:山田
main_text:鈴木一郎
score:2.6811788
-----------------------------
id:1234567
title:鈴木
main_text:None
score:1.7734823
-----------------------------
id:123456789
title:鈴木 鈴木
main_text:None
score:1.7734822
-----------------------------
id:1234567890
title:鈴木 鈴木健司
main_text:None
score:1.4480422
-----------------------------


In [64]:
search_word="鈴木 一郎"
url=g_url_base+"select?q="+search_word+"&wt=json&indent=true&defType=edismax&qf=title^1.5+main_text&tie=1&fl=*,score"
print(url)
ret=requests.post(url,headers=header)

jsonData = ret.json()["response"]["docs"]
for i in range(len(jsonData)):
    for key in jsonData[i].keys():
        data = jsonData[i][key]
        if type(data)==str:
            data=data[:50]
        if "_version_"==key:
            continue
        print("{0}:{1}".format(key,data))

    print("-----------------------------")
   

http://127.0.0.1:8983/solr/honzuki_test/select?q=鈴木 一郎&wt=json&indent=true&defType=edismax&qf=title^1.5+main_text&tie=1&fl=*,score
id:1234
title:鈴木
main_text:骨側一郎
score:2.7510214
-----------------------------
id:123456
title:山田
main_text:鈴木一郎
score:2.6811788
-----------------------------
id:1234567
title:鈴木
main_text:None
score:1.7734823
-----------------------------
id:123456789
title:鈴木 鈴木
main_text:None
score:1.7734822
-----------------------------
id:1234567890
title:鈴木 鈴木健司
main_text:None
score:1.4480422
-----------------------------


In [65]:
search_word="鈴木 一郎"
url=g_url_base+"select?q="+search_word+"&wt=json&indent=true&defType=edismax&qf=title+main_text&fl=*,score"
print(url)
ret=requests.post(url,headers=header)

jsonData = ret.json()["response"]["docs"]
for i in range(len(jsonData)):
    for key in jsonData[i].keys():
        data = jsonData[i][key]
        if type(data)==str:
            data=data[:50]
        if "_version_"==key:
            continue            
        print("{0}:{1}".format(key,data))
    print("-----------------------------")
   

http://127.0.0.1:8983/solr/honzuki_test/select?q=鈴木 一郎&wt=json&indent=true&defType=edismax&qf=title+main_text&fl=*,score
id:123456
title:山田
main_text:鈴木一郎
score:2.6811788
-----------------------------
id:1234
title:鈴木
main_text:骨側一郎
score:2.1598606
-----------------------------
id:1234567
title:鈴木
main_text:None
score:1.1823215
-----------------------------
id:123456789
title:鈴木 鈴木
main_text:None
score:1.1823215
-----------------------------
id:1234567890
title:鈴木 鈴木健司
main_text:None
score:0.9653615
-----------------------------


In [25]:
ret.json()

{'responseHeader': {'status': 0,
  'QTime': 1,
  'params': {'q.alt': '鈴木',
   'defType': 'dismax',
   'indent': 'true',
   'qf': 'main_text title',
   'fl': 'score',
   'json': '',
   'wt': 'json'}},
 'response': {'numFound': 2,
  'start': 0,
  'maxScore': 0.57735026,
  'numFoundExact': True,
  'docs': [{'score': 0.57735026}, {'score': 0.5}]}}